# AG2: Tools (Interoperability) (PydanticAI Tools Integration)

- PydanticAI Tools Integration<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/tools/interop/pydanticai

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## PydanticAI Tools Integration

### Introduction
- PydanticAI
  - https://ai.pydantic.dev/

### Installation

```python
pip install ag2[openai,interop-pydantic-ai]
```

### Imports

In [2]:
import os
from typing import Optional

from pydantic import BaseModel
from pydantic_ai import RunContext
from pydantic_ai.tools import Tool as PydanticAITool

# from autogen import AssistantAgent, UserProxyAgent, LLMConfig
from autogen import AssistantAgent, UserProxyAgent
from autogen.interop import Interoperability

### Agent Configuration

In [3]:
# config_list = [{"api_type": "openai", "model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]
config_list = {"api_type": "openai", "model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding_work",
        "use_docker": False,  # Set to True if you want to use Docker
    }
)

chatbot = AssistantAgent(
    name="chatbot",
    # llm_config=LLMConfig(config_list=config_list),
    llm_config=config_list,
)

### Tool Integration

In [4]:
class Player(BaseModel):
    name: str
    age: int


def get_player(ctx: RunContext[Player], additional_info: Optional[str] = None) -> str:  # type: ignore[valid-type]
    """Get the player's name.

    Args:
        additional_info: Additional information which can be used.
    """
    return f"Name: {ctx.deps.name}, Age: {ctx.deps.age}, Additional info: {additional_info}"  # type: ignore[attr-defined]


interop = Interoperability()
pydantic_ai_tool = PydanticAITool(get_player, takes_ctx=True)

# player will be injected as a dependency
player = Player(name="Luka", age=25)
ag2_tool = interop.convert_tool(tool=pydantic_ai_tool, type="pydanticai", deps=player)

ag2_tool.register_for_execution(user_proxy)
ag2_tool.register_for_llm(chatbot)

### Initiating the Chat

In [5]:
message = "Get player, for additional information use 'goal keeper'"

In [6]:
chat_result = user_proxy.initiate_chat(recipient=chatbot, message=message, max_turns=3)

User (to chatbot):

Get player, for additional information use 'goal keeper'

--------------------------------------------------------------------------------
chatbot (to User):

***** Suggested tool call (call_P9IMI7WI1nmOeACBJXjlydBC): get_player *****
Arguments: 
{"additional_info":"goal keeper"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_player...
Call ID: call_P9IMI7WI1nmOeACBJXjlydBC
Input arguments: {'additional_info': 'goal keeper'}
User (to chatbot):

***** Response from calling tool (call_P9IMI7WI1nmOeACBJXjlydBC) *****
Name: Luka, Age: 25, Additional info: goal keeper
**********************************************************************

--------------------------------------------------------------------------------
chatbot (to User):

The player's name is Luka, he is 25 years old, and he plays as a goal keeper.

TERMINATE

---

### Output

In [7]:
print(chat_result.summary)

In [8]:
print(len(chat_result.chat_history))
# print(chat_result.chat_history[0])
# print(chat_result.chat_history[-1])
print(chat_result.chat_history)

6
[{'content': "Get player, for additional information use 'goal keeper'", 'role': 'assistant', 'name': 'User'}, {'tool_calls': [{'id': 'call_P9IMI7WI1nmOeACBJXjlydBC', 'function': {'arguments': '{"additional_info":"goal keeper"}', 'name': 'get_player'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': 'Name: Luka, Age: 25, Additional info: goal keeper', 'tool_responses': [{'tool_call_id': 'call_P9IMI7WI1nmOeACBJXjlydBC', 'role': 'tool', 'content': 'Name: Luka, Age: 25, Additional info: goal keeper'}], 'role': 'tool', 'name': 'User'}, {'content': "The player's name is Luka, he is 25 years old, and he plays as a goal keeper.\n\nTERMINATE", 'role': 'user', 'name': 'chatbot'}, {'content': '', 'role': 'assistant', 'name': 'User'}, {'content': 'TERMINATE', 'role': 'user', 'name': 'chatbot'}]
